In [23]:
%matplotlib inline
import numpy as np
import math
import random
import pyJHTDB
import pandas as pd
import os

In [24]:
dx=8*np.pi/100
dy=0.02
dz=3*np.pi/100
x=np.zeros(100)
y=np.zeros(100)
z=np.zeros(100)
for i in range(100):
    x[i]=i*dx
    y[i]=-1+i*dy
    z[i]=i*dz
print(x[5], y[8], z[3]) #just a check on values

points1=[[None]*3]*200000
points2=[[None]*3]*200000
points3=[[None]*3]*200000
points4=[[None]*3]*200000
points5=[[None]*3]*200000
for i in range(100):
    for j in range(100):
        for k in range(20):
            points1[2000*i+20*j+k]=[x[i],y[j],z[k]]
            points2[2000*i+20*j+k]=[x[i],y[j],z[k+20]]
            points3[2000*i+20*j+k]=[x[i],y[j],z[k+40]]
            points4[2000*i+20*j+k]=[x[i],y[j],z[k+60]]
            points5[2000*i+20*j+k]=[x[i],y[j],z[k+80]]
            

In [25]:
points1 = np.array(points1,dtype='float32')
points2 = np.array(points2,dtype='float32')
points3 = np.array(points3,dtype='float32')
points4 = np.array(points4,dtype='float32')
points5 = np.array(points5,dtype='float32')
print(points1[830], points2[8300], len(points4), points5.shape[-1])


[ 0.        -0.18       0.9424778] [ 1.0053097 -0.7        1.8849556] 200000 3


In [26]:
time=np.zeros(5000)
dt=0.0065
for i in range(5000):
    time[i]=i*0.0065

In [27]:
spatialInterp  = 6  # 6 point Lagrange
temporalInterp = 0  # no time interpolation at least on these points
FD4Lag4        = 44 #for gradients
lTDB = pyJHTDB.libJHTDB()
#initialize webservices
lTDB.initialize()
auth_token  = "com.gmail.jpickett713-28046a94" #token receieved from JHU
lTDB.add_token(auth_token)

In [28]:
print(lTDB) #check that lTDB initialized

In [29]:
result1 = lTDB.getData(time[1], points1,
        sinterp = spatialInterp, tinterp = temporalInterp,data_set ='channel',
        getFunction = 'getVelocity')
result2 = lTDB.getData(time[1], points2,
        sinterp = spatialInterp, tinterp = temporalInterp,data_set ='channel',
        getFunction = 'getVelocity')
result3 = lTDB.getData(time[1], points3,
        sinterp = spatialInterp, tinterp = temporalInterp,data_set ='channel',
        getFunction = 'getVelocity')
result4 = lTDB.getData(time[1], points4,
        sinterp = spatialInterp, tinterp = temporalInterp,data_set ='channel',
        getFunction = 'getVelocity')
result5 = lTDB.getData(time[1], points5,
        sinterp = spatialInterp, tinterp = temporalInterp,data_set ='channel',
        getFunction = 'getVelocity')

In [30]:
columns=['x','y','z','u','v', 'w']
arr_1=np.zeros((200000,6))
arr_2=np.zeros((200000,6))
arr_3=np.zeros((200000,6))
arr_4=np.zeros((200000,6))
arr_5=np.zeros((200000,6))
for j in range(200000):
    arr_1[j,0]=x[int(j//2000)]
    arr_1[j,1]=y[int(j//20)%100]
    arr_1[j,2]=z[j%20]
    arr_2[j,0]=x[int(j//2000)]
    arr_2[j,1]=y[int(j//20)%100]
    arr_2[j,2]=z[20+j%20]
    arr_3[j,0]=x[int(j//2000)]
    arr_3[j,1]=y[int(j//20)%100]
    arr_3[j,2]=z[40+j%20]
    arr_4[j,0]=x[int(j//2000)]
    arr_4[j,1]=y[int(j//20)%100]
    arr_4[j,2]=z[60+j%20]
    arr_5[j,0]=x[int(j//2000)]
    arr_5[j,1]=y[int(j//20)%100]
    arr_5[j,2]=z[80+j%20]
for i in range(200000):
    arr_1[i,3:6]=result1[i]
    arr_2[i,3:6]=result2[i]
    arr_3[i,3:6]=result3[i]
    arr_4[i,3:6]=result4[i]
    arr_5[i,3:6]=result5[i]
    

In [36]:
df=pd.DataFrame(data=np.concatenate((arr_1,arr_2,arr_3,arr_4,arr_5)), columns=columns)
df1=df[['u','v','w']]

In [39]:
df.shape #check dataframe is right shape
df1.shape

(1000000, 3)

In [40]:
df1.to_csv('time_1.csv')